In [145]:
import sys
sys.path.append('../_amigocloud')

from amigocloud import AmigoCloud

import geopandas as gpd
import pandas as pd
from datetime import datetime
from datetime import timedelta

from shapely import wkb

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [109]:
path_xlsx_plan = r'C:\Documents\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\pd_trichogramma_2024 - copia.xlsx'
path_txt_msj = r'C:\Users\bismarksr\Desktop\PYTHON\utea_scripts\notificaciones_whastapp\_msj_stack.txt'

tecnico_telf = {
    'ALEJANDRO SANCHEZ' : 78448317,
    'AURELIO GARCIA' : 79051559,
    'CHRISTIAN CALIZAYA' : 63619660,
    'EDUARDO REYES' : 75016609}

## ejecutar query

In [5]:
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    # variable para almacenar resultado
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

In [143]:
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)

In [10]:
proyecto_id = 33457

In [11]:
df_plan = pd.read_excel(path_xlsx_plan, sheet_name='PLANIFICACION')

In [12]:
df_plan = df_plan[['unidad_03', 'unidad_04', 'telefono_1', 'telefono_2', 'telefono_3']]

In [13]:
df_plan.head(3)

,unidad_03,unidad_04,telefono_1,telefono_2,telefono_3
0,18358,ROJAS AMELUNGE CARLOS,77395410,0,NaN
1,18358,ROJAS AMELUNGE CARLOS,77395410,0,NaN
2,18358,ROJAS AMELUNGE CARLOS,77395410,0,NaN


In [16]:
query = 'select * from dataset_354655 where procesado=false'
notif = ejecutar_query_sql(proyecto_id, query, 'get')

In [17]:
notif = notif['data']

In [18]:
notif

[{'fecha_registro': '2024-09-11 20:01:10+00:00',
  'amigo_id': '9ff59ac965874f46bca0725c07435fb2',
  'id': 2,
  'piloto': 'ALEJANDRO SANCHEZ',
  'canhero': '0 / BISMARK SOCOMPI',
  'ubicacion': '0101000020E61000009AD99DA4AF954FC0B2CDD776244831C0',
  'tipo_labor': 'LIBERACION DE TRICHOGRAMMA',
  'tipo_mensaje': 'INICIO DE LABOR',
  'procesado': False}]

In [35]:
reg_amigo = notif[0]

In [76]:
cod_ca = int(reg_amigo['canhero'].split(' / ')[0])
nom_ca = reg_amigo['canhero'].split(' / ')[1]

In [77]:
reg = df_plan[df_plan['unidad_03']==cod_ca].iloc[0]
reg

unidad_03                     0
unidad_04     BISMARK SOCOMPI R
telefono_1             78194371
telefono_2             62022996
telefono_3                  NaN
Name: 746, dtype: object

In [78]:
idd = reg_amigo['id']
idd

2

In [79]:
fecha_hora_obj = datetime.fromisoformat(reg_amigo['fecha_registro'])
fecha_hora_menos_4h = fecha_hora_obj - timedelta(hours=4)

In [80]:
fecha_str = fecha_hora_menos_4h.strftime('%d-%m-%Y')
fecha_str

'11-09-2024'

In [81]:
hora_str = fecha_hora_menos_4h.strftime('%H:%M:%S')
hora_str

'16:01:10'

In [82]:
cod_ca

0

In [83]:
nom_ca

'BISMARK SOCOMPI'

In [93]:
telefonos = [reg['telefono_1'], reg['telefono_2'], reg['telefono_3']]
telefonos_validos = [telefono for telefono in telefonos if isinstance(telefono, int) and len(str(telefono)) == 8]

In [94]:
telefonos_validos

[78194371, 62022996]

In [101]:
labor = reg_amigo['tipo_labor']
labor

'LIBERACION DE TRICHOGRAMMA'

In [103]:
estado = reg_amigo['tipo_mensaje']
estado

'INICIO DE LABOR'

In [104]:
tecnico = reg_amigo['piloto']
tecnico

'ALEJANDRO SANCHEZ'

In [153]:
ubi = convertir_wkb(reg_amigo['ubicacion'])
longitud = ubi.x  # Longitud (coordenada X)
latitud = ubi.y   # Latitud (coordenada Y)

shapely.geometry.point.Point

In [206]:
def crear_msj(idd, fecha, hora, cod, nombre, telfs, labor, estado, tecnico, lon, lat):
    tec_telf = tecnico_telf[tecnico]
    for i in telfs:
        msj = ''
        if estado == 'INICIO DE LABOR':
            msj = f'''Señor(a) *{nombre}* se le informa que se esta iniciando el trabajo de *{labor}* en la siguiente ubicacion: \
https://www.google.com/maps?q={lat},{lon}. \
Para cualquier consulta puede comunicarse con ING. {tecnico} al {tec_telf}'''
        elif estado == 'FIN DE LABOR':
            msj = f'''Señor(a) *{nombre}* se le informa que se ha finalizado el trabajo de *{labor}* en la siguiente ubicacion: \
https://www.google.com/maps?q={lat},{lon}. \
Para cualquier consulta puede comunicarse con ING. {tecnico} al {tec_telf}'''
        linea = f'{idd};{fecha};{hora};{cod};{nombre};{i};{msj}'
        #linea2 = f'{idd};{fecha};{hora};{cod};{nombre};{i};https://www.google.com/maps?q={lat},{lon}'
        with open(path_txt_msj, 'r') as file:
            contenido = file.readlines()
        contenido.insert(0, '\n')
        contenido.insert(0, linea)
        #contenido.insert(0, '\n')
        #contenido.insert(0, linea2)
        with open(path_txt_msj, 'w') as file:
            file.writelines(contenido)
        print(linea)

In [207]:
crear_msj(idd, fecha_str, hora_str, cod_ca, nom_ca, telefonos_validos, labor, estado, tecnico, longitud, latitud)

2;11-09-2024;16:01:10;0;BISMARK SOCOMPI;78194371;Señor(a) *BISMARK SOCOMPI* se le informa que se esta iniciando el trabajo de *LIBERACION DE TRICHOGRAMMA* en la siguiente ubicacion: https://www.google.com/maps?q=-17.2818064,-63.1694227. Para cualquier consulta puede comunicarse con ING. ALEJANDRO SANCHEZ al 78448317
2;11-09-2024;16:01:10;0;BISMARK SOCOMPI;62022996;Señor(a) *BISMARK SOCOMPI* se le informa que se esta iniciando el trabajo de *LIBERACION DE TRICHOGRAMMA* en la siguiente ubicacion: https://www.google.com/maps?q=-17.2818064,-63.1694227. Para cualquier consulta puede comunicarse con ING. ALEJANDRO SANCHEZ al 78448317
